# Assignment 2A

**Name**:  Adwik Gupta                             </br>
**Roll No.**: 220085

In [1]:
# import
import gymnasium as gym
import copy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
import random

In [27]:
'''
Represents a Stochastic Maze problem Gym Environment which provides a Fully observable
MDP
'''
class StochasticMazeEnv(gym.Env):
    '''
    StochasticMazeEnv represents the Gym Environment for the Stochastic Maze environment
    States : [0,1,2,3,4,5,6,7,8,9,10,11]
    Actions : ["Left":0, "Up":1, "Right":2, "Down":3]
    '''
    metadata = {'render.modes': ['human']}

    def __init__(self,initial_state=0,no_states=12,no_actions=4):
        '''
        Constructor for the StochasticMazeEnv class

        Args:
            initial_state : starting state of the agent
            no_states : The no. of possible states which is 12
            no_actions : The no. of possible actions which is 4
            
        '''
        self.initial_state = initial_state
        self.state = self.initial_state
        self.nA = no_actions
        self.nS = no_states
        self.actions_dict = {"L":0, "U":1, "R":2, "D":3}
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        self.reset()

    def reset(self):
        '''
        Resets the environment
        Returns:
            observations containing player's current state
        '''
        self.state = self.initial_state
        return self.get_obs()

    def get_obs(self):
        '''
        Returns the player's state as the observation of the environment
        '''
        return (self.state)

    def render(self, mode='human'):
        '''
        Renders the environment
        '''
        print("Current state: {}".format(self.state))

    def sample_action(self):
        '''
        Samples and returns a random action from the action space
        '''
        return random.randint(0, self.nA-1)
    def P(self):
        '''
        Defines and returns the probabilty transition matrix which is in the form of a nested dictionary
        '''
        self.prob_dynamics = {
            0: {
                0: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
                2: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 1, -0.01, False)],
            },
            1: {
                0: [(0.8, 0, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                1: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
                2: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 1, -0.01, False)],
                3: [(0.8, 1, -0.01, False), (0.1, 0, -0.01, False), (0.1, 2, -0.01, False)],
            },
            2: {
                0: [(0.8, 1, -0.01, False), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
                2: [(0.8, 3, +1, True), (0.1, 2, -0.01, False), (0.1, 6, -0.01, False)],
                3: [(0.8, 6, -0.01, False), (0.1, 1, -0.01, False), (0.1, 3, +1, True)],
            },
            3: {
                0: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                1: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                2: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
                3: [(0.8, 3, 0, True), (0.1, 3, 0, True), (0.1, 3, 0, True)],
            },
            4: {
                0: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                1: [(0.8, 0, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
                2: [(0.8, 4, -0.01, False), (0.1, 0, -0.01, False), (0.1, 8, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 4, -0.01, False), (0.1, 4, -0.01, False)],
            },
            6: {
                0: [(0.8, 6, -0.01, False), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 2, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
                2: [(0.8, 7, -1, True), (0.1, 2, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 6, -0.01, False), (0.1, 7, -1, True)],
            },
            7: {
                0: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                1: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                2: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
                3: [(0.8, 7, 0, True), (0.1, 7, 0, True), (0.1, 7, 0, True)],
            },
            8: {
                0: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                1: [(0.8, 4, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
                2: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 4, -0.01, False)],
                3: [(0.8, 8, -0.01, False), (0.1, 8, -0.01, False), (0.1, 9, -0.01, False)],
            },
            9: {
                0: [(0.8, 8, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                1: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)],
                2: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 9, -0.01, False)],
                3: [(0.8, 9, -0.01, False), (0.1, 8, -0.01, False), (0.1, 10, -0.01, False)]
            },
            10: {
                0: [(0.8, 9, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                1: [(0.8, 6, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 6, -0.01, False), (0.1, 10, -0.01, False)],
                3: [(0.8, 10, -0.01, False), (0.1, 9, -0.01, False), (0.1, 11, -0.01, False)]
            },
            11: {
                0: [(0.8, 10, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                1: [(0.8, 7, -1, True), (0.1, 10, -0.01, False), (0.1, 11, -0.01, False)],
                2: [(0.8, 11, -0.01, False), (0.1, 7, -1, True), (0.1, 11, -0.01, False)],
                3: [(0.8, 11, -0.01, False), (0.1, 11, -0.01, False), (0.1, 10, -0.01, False)]
            }
        }
        return self.prob_dynamics


    def step(self, action):
        '''
        Performs the given action
        Args:
            action : action from the action_space to be taking in the environment
        Returns:
            observation - returns current state
            reward - reward obtained after taking the given action
            done - True if the episode is complete else False
        '''
        if action >= self.nA:
            action = self.nA-1

        index = np.random.choice(3,1,p=[0.8,0.1,0.1])[0]

        dynamics_tuple = self.prob_dynamics[self.state][action][index]
        self.state = dynamics_tuple[1]
        

        return self.state, dynamics_tuple[2], dynamics_tuple[3]

In [28]:
env = StochasticMazeEnv()
env.reset()
num_states = env.nS
num_actions = env.nA

## Test Cases for checking the environment 

## Example: Random Policy

In [29]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1

    rand_action = np.random.choice(4,1)[0]  #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(rand_action)
    
    total_reward += reward

    print("  ", rand_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   0 	   0 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   1 	   0 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   2 	   0 	 -0.01 	   False
   0 	   0 	 -0.01 	   False
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   3 	   3 	 1 	   True
Total Number of steps to Reach Terminal: 10
Final Reward: 0.91


### The random policy takes large number of steps to reach some terminal state which should be much higher than the number of the steps taken by a all 'Right' policy

## Example: Right Policy

In [30]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1

    right_action = 2  #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(right_action)
    
    total_reward += reward

    print("  ", right_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   2 	   3 	 1 	   True
Total Number of steps to Reach Terminal: 3
Final Reward: 0.98


### The right policy most of the time reaches the goal state in just 3 steps which is expected

***

## Q1. Find an optimal policy to navigate the given environment using Policy Iteration 

In [143]:
# write your code here

env.reset()

# setting the default values of V  of all states

V = [-0.01,-0.01,-0.01,1,-0.01,-0.01,-1,-0.01,-0.01,-0.01,-0.01]
print(V)

# setting the default policy

pi = [env.sample_action() ,env.sample_action() ,env.sample_action() ,env.sample_action() ,env.sample_action() ,env.sample_action() ,env.sample_action() ,env.sample_action() ,env.sample_action() ,env.sample_action() ,env.sample_action()]
pi_copy = pi.copy()
#pi = [2]*11
print(pi)

# hint: code policy evaluation function

def policy_eval(value,policy):
    states = [0,1,2,3,4,6,7,8,9,10,11]
    val = value.copy()
    for j in range(11):
        action_set = env.P()[states[j]][policy[j]]
        for action in action_set:
            if(action[1]<5):
                value[j] += action[0]*(action[2]+val[action[1]])
            else:
                value[j] += action[0]*(action[2]+val[action[1]-1])               
    print("value ex",value)            
    return value

# hint: code policy improvement function

def policy_improvement(value,policy):
    states = [0,1,2,3,4,6,7,8,9,10,11]
    for j in states:
        v = [0]*4
        path_set = env.P()[j]
        #for i,path in enumerate(path_set):
        for i in range(4):
            for path_in in path_set[i]:
                if path_in[1]<5:
                    v[i] += path_in[0]*(path_in[2]+value[path_in[1]])
                else:
                    v[i] += path_in[0]*(path_in[2]+value[path_in[1]-1])
        #print(j,v)            
        if len(set(v)) != 1:
            max_index = v.index(max(v))
            if j<5:
                policy[j] = max_index
            else:
                policy[j-1] = max_index  
        #print("policy",policy)             
    return policy         

# hint: implement policy iteration
count = 0

while(1):
    count += 1
    pi_dummy = pi.copy()
    V = policy_eval(V,pi)
    pi = policy_improvement(V,pi)
    if pi_dummy == pi:
        break

print(pi)
print(count)


[-0.01, -0.01, -0.01, 1, -0.01, -0.01, -1, -0.01, -0.01, -0.01, -0.01]
[0, 3, 3, 1, 0, 2, 3, 2, 3, 3, 1]
value ex [-0.030000000000000006, -0.030000000000000006, 0.17200000000000001, 2.0, -0.030000000000000006, -1.614, -2.0, -0.030000000000000006, -0.030000000000000006, -0.030000000000000006, -1.614]
value ex [-0.07000000000000002, 0.09160000000000001, 2.4258000000000006, 4.0, -0.07000000000000002, -1.9468000000000003, -4.0, -0.07000000000000002, -0.07000000000000002, -0.22840000000000005, -2.1084]
value ex [-0.020720000000000006, 2.040560000000001, 6.4717, 8.0, -0.15000000000000005, -0.7098399999999996, -8.0, -0.15000000000000005, -0.15000000000000005, -0.51192, -3.0109600000000003]
value ex [1.584656000000001, 7.616032000000002, 14.245886, 16.0, -0.2065760000000001, 3.4875360000000017, -16.0, -0.31000000000000016, -0.31000000000000016, -0.764096, -4.630592]
value ex [7.805289600000003, 20.525947200000004, 29.6172282, 32.0, 1.0098336000000008, 13.523998400000004, -32.0, -0.547260800000

In [146]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

states = [0,1,2,3,4,6,7,8,9,10,11]
state_action = [[states[i], pi[i]] for i in range(11)]
state = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1
    
    right_action = state_action[state][1] if state<5 else state_action[state-1][1]   #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(right_action)
    
    total_reward += reward

    print("  ", right_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   2 	   3 	 1 	   True
Total Number of steps to Reach Terminal: 3
Final Reward: 0.98


## Q2. Find an optimal policy to navigate the given environment using Value Iteration

In [148]:
# write your code here
V1 = [-0.01,-0.01,-0.01,1,-0.01,-0.01,-1,-0.01,-0.01,-0.01,-0.01]
def value_iteration(value):
    states = [0,1,2,3,4,6,7,8,9,10,11]
    for j in states:
        v = [0]*4
        path_set = env.P()[j]
        for i in range(4):
            for path_in in path_set[i]:
                if path_in[1]<5:
                    v[i] += path_in[0]*(path_in[2]+value[path_in[1]])
                else:
                    v[i] += path_in[0]*(path_in[2]+value[path_in[1]-1])
        if j<5:
            value[j] = max(v)
        else:
            value[j-1] = max(v)
    print(value)
    return value
c = 0
while(1):
    c += 1
    V2 = V1.copy()
    V1 = value_iteration(V1)
    if all(abs(v2_i - v1_i) < 0.01 for v2_i, v1_i in zip(V2, V1)):
        break
print(V1)    
print(c)

def policy_finder(value,policy):
    states = [0,1,2,3,4,6,7,8,9,10,11]
    for j in states:
        v = [0]*4
        path_set = env.P()[j]
        #for i,path in enumerate(path_set):
        for i in range(4):
            for path_in in path_set[i]:
                if path_in[1]<5:
                    v[i] += path_in[0]*(path_in[2]+value[path_in[1]])
                else:
                    v[i] += path_in[0]*(path_in[2]+value[path_in[1]-1])
        #print(j,v)            
        if len(set(v)) != 1:
            max_index = v.index(max(v))
            if j<5:
                policy[j] = max_index
            else:
                policy[j-1] = max_index  
        #print("policy",policy)             
    return policy    

print("pi",pi_copy)
pi_vi = policy_finder(V1,pi_copy)
print(pi_vi) 

        

[-0.020000000000000004, -0.020000000000000004, 1.596, 1.0, -0.020000000000000004, 1.0668000000000002, -1.0, -0.020000000000000004, -0.020000000000000004, 0.8404400000000002, 0.46235200000000015]
[-0.030000000000000013, 1.2628000000000004, 1.8642800000000002, 1.0, -0.030000000000000013, 1.3891040000000003, -1.0, -0.030000000000000013, 0.6583520000000002, 1.2133536000000005, 0.8079180800000005]
[0.9942400000000002, 1.7339840000000004, 1.9233384, 1.0, 0.7793920000000002, 1.46858112, -1.0, 0.6763488000000002, 1.0923532800000006, 1.3548920320000002, 0.9557054336000003]
[1.5545504000000006, 1.8754675200000002, 1.937191952, 1.0, 1.3895187200000005, 1.4940732944000001, -1.0, 1.2784851840000007, 1.2923842816000004, 1.4100676070400002, 1.0146246289920002]
[1.7847809280000004, 1.9148470656, 1.94112652464, 1.0, 1.6957284864000006, 1.520378048688, -1.0, 1.6036697356800007, 1.5314126448640006, 1.508174681464001, 1.099002208070401]
[1.8699285939200003, 1.9258706328319999, 1.9441504573328001, 1.0, 1.8

In [149]:
is_Terminal = False
env.reset()
count = 0
total_reward = 0

states = [0,1,2,3,4,6,7,8,9,10,11]
state_action = [[states[i], pi_vi[i]] for i in range(11)]
state = 0

print("Action\t" , "State\t" , "Reward\t" , "is_Terminal")

while not is_Terminal:
    count += 1
    
    right_action = state_action[state][1] if state<5 else state_action[state-1][1]   #0 -> LEFT, 1 -> UP, 2 -> RIGHT, 3 -> DOWN
    state, reward, is_Terminal = env.step(right_action)
    
    total_reward += reward

    print("  ", right_action, "\t  ", state, "\t", reward, "\t  ", is_Terminal)
    
print("Total Number of steps to Reach Terminal:", count)
print("Final Reward:", total_reward)

Action	 State	 Reward	 is_Terminal
   2 	   1 	 -0.01 	   False
   2 	   2 	 -0.01 	   False
   2 	   3 	 1 	   True
Total Number of steps to Reach Terminal: 3
Final Reward: 0.98


## Q3. Compare PI and VI in terms of convergence. Is the policy obtained by both same?

### Answer: 
In this case the policy obtained is not entirely same but both are leading to optimal solutions.
VI may take more iterations, but each iteration tends to be less computationally expensive, as it performs both evaluation and improvement simultaneously.